In [ ]:
!nvidia-smi

Tue Sep  3 08:49:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#check Tensorflow version
!pip show tensorflow

Name: tensorflow
Version: 2.17.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras


In [ ]:
#downgrade the version of tensorflow
!pip install --upgrade tensorflow==2.16.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installation

In [ ]:
cd /content/drive/MyDrive/NLP_PROJECT/

/content/drive/MyDrive/NLP_PROJECT


In [ ]:
# import the necessery packages
import numpy as np
import tensorflow as tf
#from validation import compute_f1
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional, GlobalMaxPool1D, MaxPooling1D,Flatten,concatenate
#from prepro import readfile, createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from tensorflow.keras.utils import Progbar
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import RandomUniform
import statistics
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Attention #Attention lauer
#from keras_crf import CRFModel # Conditional Random Field
#from keras_contrib.layers import CRF
#from tensorflow_addons.layers import CRF # tfa = Tensorflow addon
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

In [ ]:
#prepro.py
import numpy as np
import random
from tensorflow.keras.preprocessing.sequence import pad_sequences

# read a text file and extract the sentences
def readfile(filename):
    '''
    read file
    return format :
    [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]
    '''
    f = open(filename)
    sentences = []
    sentence = []
    for line in f:
        if len(line)==0 or line.startswith('-DOCSTART') or line[0]=="\n":
            if len(sentence) > 0:
                sentences.append(sentence)
                sentence = []
            continue
        splits = line.split(' ')
        sentence.append([splits[0],splits[-1].replace("\n","")])

    if len(sentence) >0:
        sentences.append(sentence)
        sentence = []
    return sentences

# Extract the casing status from the word
def getCasing(word, caseLookup):
    casing = 'other'

    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1

    digitFraction = numDigits / float(len(word))

    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'


    return caseLookup[casing]

# Divide the dataset into mini-batches to be trained on the model
def createBatches(data):
    l = []
    for i in data:
        l.append(len(i[0]))
    l = set(l)
    batches = []
    batch_len = []
    z = 0
    for i in l:
        for batch in data:
            if len(batch[0]) == i:
                batches.append(batch)
                z += 1
        batch_len.append(z)
    return batches,batch_len

def createMatrices(sentences, pos,word2Idx, label2Idx, case2Idx,char2Idx):
    unknownIdx = word2Idx['UNKNOWN_TOKEN']
    paddingIdx = word2Idx['PADDING_TOKEN']

    dataset = []

    wordCount = 0
    unknownWordCount = 0
    i=0
    for sentence in sentences:
        wordIndices = []
        caseIndices = []
        charIndices = []
        labelIndices = []

        for word,char,label in sentence:
            wordCount += 1
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]
            else:
                wordIdx = unknownIdx
                unknownWordCount += 1
            charIdx = []
            for x in char:
                charIdx.append(char2Idx[x])

            #Get the label and map to int
            wordIndices.append(wordIdx)
            caseIndices.append(getCasing(word, case2Idx))
            charIndices.append(charIdx)
            labelIndices.append(label2Idx[label])

        dataset.append([wordIndices,pos[i], caseIndices, charIndices, labelIndices])
        i+=1

    return dataset


# Iterate through mini-batches
def iterate_minibatches(dataset,batch_len):
    start = 0
    for i in batch_len:
        tokens = []
        pos=[]
        caseing = []
        char = []
        labels = []
        data = dataset[start:i]
        start = i
        for dt in data:
            t,p,c,ch,l = dt
            l = np.expand_dims(l,-1)
            tokens.append(t)
            pos.append(p)
            caseing.append(c)
            char.append(ch)
            labels.append(l)
        yield np.asarray(labels),np.asarray(tokens),np.asarray(pos), np.asarray(caseing),np.asarray(char)

# split each word into an array of characters
def addCharInformatioin(Sentences):
    for i,sentence in enumerate(Sentences):
        for j,data in enumerate(sentence):
            chars = [c for c in data[0]]
            Sentences[i][j] = [data[0],chars,data[1]]
    return Sentences

def padding(Sentences):
    maxlen = 52
    for sentence in Sentences:
        char = sentence[3]
        for x in char:
            maxlen = max(maxlen,len(x))
    for i,sentence in enumerate(Sentences):
        Sentences[i][3] = pad_sequences(Sentences[i][3],52,padding='post')
    return Sentences


In [ ]:
#Read the text file
data= readfile("/content/drive/MyDrive/NLP_PROJECT/data/dataset.txt")

In [ ]:
# split the dataset to training set and validation set

from sklearn.model_selection import train_test_split

trainSentences, testSentences = train_test_split(data, test_size=0.2, random_state=15)
#trainSentences, validSentences = train_test_split(trainSentences, test_size=0.1, random_state=15)

In [ ]:
print(f"No. of training examples: {len(trainSentences)}")
#print(f"No. of training examples: {len(validSentences)}")
print(f"No. of testing examples: {len(testSentences)}")

No. of training examples: 15364
No. of testing examples: 3841


In [ ]:
#ex: [['We', ['W', 'e'], 'O'],  ['pay', ['p', 'a', 'y'], 'O'],  ['for', ['f', 'o', 'r'], 'O']]
trainSentences = addCharInformatioin(trainSentences)
#validSentences = addCharInformatioin(validSentences)
testSentences = addCharInformatioin(testSentences)
#ttt#testSen = addCharInformatioin(list(test_sen["E"]))



In [ ]:
#read train and test datasets as a dataframe in order to extract the POS tag
def readfile_(filename):
    '''
    read file
    return format :
    [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]
    '''
    f = open(filename)
    sentences = []
    sentence = []
    number=1;
    for line in f:
        if len(line)==0 or line.startswith('-DOCSTART') or line[0]=="\n":
            if len(sentence) > 0:
                sentences.append(sentence)
                number+=1
                sentence = []
            continue
        splits = line.split(' ')
        sentence.append([number,splits[0],splits[-1].replace("\n","")])

    if len(sentence) >0:
        sentences.append(sentence)
        sentence = []
    return sentences
TrS = readfile_("/content/drive/MyDrive/NLP_PROJECT/data/dataset.txt")
#teS = readfile_("../data/test-all.txt")
import pandas as pd
import itertools
df = pd.DataFrame(itertools.chain.from_iterable(TrS),columns=['sentence','word','tag'])
#df_test = pd.DataFrame(itertools.chain.from_iterable(teS),columns=['sentence','word','tag'])
import nltk
nltk.download('averaged_perceptron_tagger')
#nltk.pos_tag(df_train['word'])
df['pos'] = pd.Series(nltk.pos_tag(df['word'])).apply(lambda x: x[1])
#df_test['pos'] = pd.Series(nltk.pos_tag(df_test['word'])).apply(lambda x: x[1])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
df.head()

,sentence,word,tag,pos
0,1,Houston,B-Location,NNP
1,1,Wednesday,B-Date,NNP
2,1,flood,B-Floods,VBD
3,1,storm,B-NaturalHazards,NN
4,1,killed,B-Death_And_Toll,VBN


In [ ]:
# number of each class entity in the dataset
df["tag"].value_counts()

,count
tag,
O,397102
B-Location,9410
B-Date,4944
B-NaturalHazards,4501
I-Location,3123
I-Date,2546
B-Floods,1905
I-Death_And_Toll,1368
I-AffectedPopulation,1150


In [ ]:
# Convert POS from categorical to numbers
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
df['pos_code'] = LE.fit_transform(df['pos'])
#ttt#df1['pos_code'] = LE.fit_transform(df1['POS'])

In [ ]:
df.head()

,sentence,word,tag,pos,pos_code
0,1,Houston,B-Location,NNP,19
1,1,Wednesday,B-Date,NNP,19
2,1,flood,B-Floods,VBD,34
3,1,storm,B-NaturalHazards,NN,18
4,1,killed,B-Death_And_Toll,VBN,36


In [ ]:
# create a list for pos_code for each sentence
pos_sen=df.groupby('sentence')['pos_code'].apply(list)
#ttt#pos_sen_test=df1.groupby('sentence')['pos_code'].apply(list)

In [ ]:
## split pos tag feature
from sklearn.model_selection import train_test_split

trainposSentences, testposSentences = train_test_split(pos_sen, test_size=0.2, random_state=15)
#trainposSentences, validposSentences = train_test_split(trainposSentences, test_size=0.1, random_state=15)

In [ ]:
# start indexs from 0
trainposSentences = trainposSentences.reset_index(drop=True)
#validposSentences=validposSentences.reset_index(drop=True)
testposSentences=testposSentences.reset_index(drop=True)
#tt#testp=pos_sen_test.reset_index(drop=True)

In [ ]:
# :: Create a mapping for the pos ::
POSEmbeddings = np.zeros((len(pos_sen),100))
i=0;
for sentence in pos_sen:
        for c in sentence:
            POSEmbeddings[i][c]=1
        i+=1

#POSEmbeddings_test = np.zeros((len(testp),100))
#i=0;
#for sentence in testp:
#        for c in sentence:
#            POSEmbeddings_test[i][c]=1
#        i+=1

In [ ]:
#for dataset in [trainSentences,validSentences, testSentences]:
labelSet = set()
words = {}
for dataset in [trainSentences, testSentences]:

    for sentence in dataset:
        for token,char,label in sentence:
            labelSet.add(label)
            words[token.lower()] = True


In [ ]:
#If we are using crf model we have to create a hotencoding
label2Idx_ = {}
labelSet=sorted(labelSet)
for label in labelSet:
    ar=np.zeros(len(labelSet))
    ar[len(label2Idx_)]=1
    label2Idx_[label] = ar

label2Idx=label2Idx_

In [ ]:
# :: Create a mapping for the labels ::
label2IdxS = {}
for label in labelSet:
    label2IdxS[label] = len(label2IdxS)
#label2Idx=label2IdxS

In [ ]:
label2IdxS

{'.O': 0,
 'B-AffectedPopulation': 1,
 'B-CollapsedStructure': 2,
 'B-Date': 3,
 'B-Death_And_Toll': 4,
 'B-Fire': 5,
 'B-Floods': 6,
 'B-InfrastructureDamage': 7,
 'B-Location': 8,
 'B-MissingPersons': 9,
 'B-NaturalHazards': 10,
 'B-PowerOutage': 11,
 'B-RoadBlocked': 12,
 'B-WaterShortage': 13,
 'B-WaterShrotage': 14,
 'I-AffectedPopulation': 15,
 'I-CollapsedStructure': 16,
 'I-Date': 17,
 'I-Death_And_Toll': 18,
 'I-Floods': 19,
 'I-InfrastructureDamage': 20,
 'I-Location': 21,
 'I-MissingPersons': 22,
 'I-NaturalHazards': 23,
 'I-PowerOutage': 24,
 'I-RoadBlocked': 25,
 'I-WaterShortage': 26,
 'I-WaterShrotage': 27,
 'O': 28}

In [ ]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')


In [ ]:
# :: Read in word embeddings ::
word2Idx = {}
wordEmbeddings = []
# reading the word embedding file in text format
# pre-trained word vectors using in-domain contexual training
fEmbeddings = open("data/vectors.100d.txt", encoding="utf-8")#contextual embedding
#fEmbeddings = open("data/glove.6B.100d.txt", encoding="utf-8")#glove embedding
#fEmbeddings = open("data/Word_2_vector.txt", encoding="utf-8")#word2vec embedding
# pre-trained word vectors from word2vec embedding
#fEmbeddings = open("embeddings/GoogleNews-vectors-negative300d.bin", 'rb')

for line in fEmbeddings:
    split = line.strip().split(" ")
    word = split[0].lower()

    if len(word2Idx) == 0: #Add padding+unknown
        word2Idx["PADDING_TOKEN"] = len(word2Idx)
        vector = np.zeros(len(split)-1) #Zero vector vor 'PADDING' word
        wordEmbeddings.append(vector)

        word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
        vector = np.random.uniform(-0.25, 0.25, len(split)-1)
        wordEmbeddings.append(vector)

    if split[0] in words:
        vector = np.array([float(num) for num in split[1:]])
        wordEmbeddings.append(vector)
        word2Idx[split[0]] = len(word2Idx)


In [ ]:
#only apply it when using contextual vector
del wordEmbeddings[-1]

In [ ]:
wordEmbeddings=np.stack( wordEmbeddings, axis=0 )

wordEmbeddings.shape

(12853, 100)

In [ ]:
char2Idx = {"PADDING":0, "UNKNOWN":1}
for c in " 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,-_()[]{}!?:;#'\"/\\%$`&=*+@^~|“”µ\n©‚£°\xad،¢–’—ƒ„":
    char2Idx[c] = len(char2Idx)

In [ ]:
train_set = padding(createMatrices(trainSentences,trainposSentences,word2Idx,  label2Idx, case2Idx,char2Idx))
#valid_set = padding(createMatrices(validSentences,validposSentences, word2Idx, label2Idx, case2Idx,char2Idx))
test_set = padding(createMatrices(testSentences,testposSentences, word2Idx, label2Idx, case2Idx,char2Idx))


In [ ]:
train_set[0]

[[173, 91, 69, 1354, 87, 36, 33, 563, 6, 75, 204, 2, 179, 5570, 8, 2, 5347, 4],
 [9, 9, 21, 19, 37, 26, 33, 26, 31, 14, 13, 10, 14, 18, 13, 10, 18, 6],
 [0, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4],
 array([[ 4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0],
        [25, 21, 24, 24, 21, 27, 26,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0],
        [28, 17, 27, 28, 24, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0],
        [95,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,

In [ ]:
idx2Label = {v: k for k, v in label2IdxS.items()}
#np.save("models/idx2Label.npy",idx2Label)
#np.save("models/word2Idx.npy",word2Idx)

In [ ]:
idx2Label

{0: '.O',
 1: 'B-AffectedPopulation',
 2: 'B-CollapsedStructure',
 3: 'B-Date',
 4: 'B-Death_And_Toll',
 5: 'B-Fire',
 6: 'B-Floods',
 7: 'B-InfrastructureDamage',
 8: 'B-Location',
 9: 'B-MissingPersons',
 10: 'B-NaturalHazards',
 11: 'B-PowerOutage',
 12: 'B-RoadBlocked',
 13: 'B-WaterShortage',
 14: 'B-WaterShrotage',
 15: 'I-AffectedPopulation',
 16: 'I-CollapsedStructure',
 17: 'I-Date',
 18: 'I-Death_And_Toll',
 19: 'I-Floods',
 20: 'I-InfrastructureDamage',
 21: 'I-Location',
 22: 'I-MissingPersons',
 23: 'I-NaturalHazards',
 24: 'I-PowerOutage',
 25: 'I-RoadBlocked',
 26: 'I-WaterShortage',
 27: 'I-WaterShrotage',
 28: 'O'}

In [ ]:
train_batch,train_batch_len = createBatches(train_set)
#valid_batch,valid_batch_len = createBatches(valid_set)
test_batch,test_batch_len = createBatches(test_set)
#ttt#test_batch1,test_batch_len1 = createBatches(test_set_out)

In [ ]:
#package is needed to run CRF class
!pip install tensorflow_addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 29.7 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.3.0
    Uninstalling typeguard-4.3.0:
      Successfully uninstalled typeguard-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.3.1 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
#CRF Class
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
from tensorflow_addons.text import crf_log_likelihood, crf_decode


class CRF(L.Layer):
    def __init__(self,
                 output_dim,
                 sparse_target=True,
                 **kwargs):
        """
        Args:
            output_dim (int): the number of labels to tag each temporal input.
            sparse_target (bool): whether the the ground-truth label represented in one-hot.
        Input shape:
            (batch_size, sentence length, output_dim)
        Output shape:
            (batch_size, sentence length, output_dim)
        """
        super(CRF, self).__init__(**kwargs)
        self.output_dim = int(output_dim)
        self.sparse_target = sparse_target
        self.input_spec = L.InputSpec(min_ndim=3)
        self.supports_masking = False
        self.sequence_lengths = None
        self.transitions = None

    def build(self, input_shape):
        assert len(input_shape) == 3
        f_shape = tf.TensorShape(input_shape)
        input_spec = L.InputSpec(min_ndim=3, axes={-1: f_shape[-1]})

        if f_shape[-1] is None:
            raise ValueError('The last dimension of the inputs to `CRF` '
                             'should be defined. Found `None`.')
        if f_shape[-1] != self.output_dim:
            raise ValueError('The last dimension of the input shape must be equal to output'
                             ' shape. Use a linear layer if needed.')
        self.input_spec = input_spec
        self.transitions = self.add_weight(name='transitions',
                                           shape=[self.output_dim, self.output_dim],
                                           initializer='glorot_uniform',
                                           trainable=True)
        self.built = True

    def compute_mask(self, inputs, mask=None):
        # Just pass the received mask from previous layer, to the next layer or
        # manipulate it if this layer changes the shape of the input
        return mask

    def call(self, inputs, sequence_lengths=None, training=None, **kwargs):
        sequences = tf.convert_to_tensor(inputs, dtype=self.dtype)
        if sequence_lengths is not None:
            assert len(sequence_lengths.shape) == 2
            assert tf.convert_to_tensor(sequence_lengths).dtype == 'int32'
            seq_len_shape = tf.convert_to_tensor(sequence_lengths).get_shape().as_list()
            assert seq_len_shape[1] == 1
            self.sequence_lengths = K.flatten(sequence_lengths)
        else:
            self.sequence_lengths = tf.ones(tf.shape(inputs)[0], dtype=tf.int32) * (
                tf.shape(inputs)[1]
            )

        viterbi_sequence, _ = crf_decode(sequences,
                                         self.transitions,
                                         self.sequence_lengths)
        output = K.one_hot(viterbi_sequence, self.output_dim)
        return K.in_train_phase(sequences, output)

    @property
    def loss(self):
        def crf_loss(y_true, y_pred):
            y_pred = tf.convert_to_tensor(y_pred, dtype=self.dtype)
            log_likelihood, self.transitions = crf_log_likelihood(
                y_pred,
                tf.cast(K.argmax(y_true), dtype=tf.int32) if self.sparse_target else y_true,
                self.sequence_lengths,
                transition_params=self.transitions,
            )
            return tf.reduce_mean(-log_likelihood)
        return crf_loss

    @property
    def accuracy(self):
        def viterbi_accuracy(y_true, y_pred):
            # -1e10 to avoid zero at sum(mask)
            mask = K.cast(
                K.all(K.greater(y_pred, -1e10), axis=2), K.floatx())
            shape = tf.shape(y_pred)
            sequence_lengths = tf.ones(shape[0], dtype=tf.int32) * (shape[1])
            y_pred, _ = crf_decode(y_pred, self.transitions, sequence_lengths)
            if self.sparse_target:
                y_true = K.argmax(y_true, 2)
            y_pred = K.cast(y_pred, 'int32')
            y_true = K.cast(y_true, 'int32')
            corrects = K.cast(K.equal(y_true, y_pred), K.floatx())
            return K.sum(corrects * mask) / K.sum(mask)
        return viterbi_accuracy

    def compute_output_shape(self, input_shape):
        tf.TensorShape(input_shape).assert_has_rank(3)
        return input_shape[:2] + (self.output_dim,)

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'sparse_target': self.sparse_target,
            'supports_masking': self.supports_masking,
            'transitions': K.eval(self.transitions)
        }
        base_config = super(CRF, self).get_config()
        return dict(base_config, **config)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
###Character Embedding

from numpy.core.fromnumeric import nonzero
from nltk.grammar import Nonterminal
# character embedding
'''
Character Embedding

1. Define a list a characters (i.e. m). For example, can use alphanumeric and some special characters. For my example, English characters (52), number (10), special characters (20) and one unknown character, UNK. Total 83 characters.
2. Transfer characters as 1-hot encoding and got a sequence for vectors. For unknown characters and blank characters, use all-zero vector to replace it. If exceeding pre-defined maximum length of characters (i.e. l), ignoring it.
The output is 30 dimensions vector per every single character.
3. Using 3 1D CNN layers (configurable) to learn the sequence
'''

#filters=[[3, 5, 7], [200, 300, 300], [300, 400, 400]]
# filters=[[100, 200, 200], [200, 300, 300], [300, 400, 400]],
#kernel_sizes=[[4, 3, 3], [5, 3, 3], [6, 3, 3]]
#pool_sizes=[[2, 2, 2], [2, 2, 2], [2, 2, 2]]

filters=[33, 33, 34]
kernel_sizes=[4, 3, 3]
pool_sizes=[2, 2, 2]
padding='same'
activation='relu'
kernel_initializer='glorot_normal'
dropout = 0.3

# number of characters is 110: "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,-_()[]{}!?:;#'\"/\\%$`&=*+@^~|“”µ\n©‚£°\xad،¢–’—ƒ„
#sent_input = Input(shape=(max_len_of_sentence, ), dtype='int64')
#embedded = Embedding(self.num_of_char, char_dimension, input_length=max_len_of_sentence)(sent_input)
character_input=Input(shape=(None,52,),dtype='int64', name='char_input')
# Embedding input: num_char, output vector dimension, input_length, embedding_initializer, name
# random initialization of a lookup table with values drawn from a uniform distribution with range [-0.5, 0.5] to output a character embedding of 25 dimension
embed_char_out=TimeDistributed(Embedding(len(char2Idx),100,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
#embed_char_out= Dropout(0.25)(embed_char_out)
#for i, filter_layers in enumerate(filters):
print(embed_char_out.shape)
blocks=[]
block = embed_char_out
for i, filter_layers in enumerate(filters):

            block = TimeDistributed(Conv1D(
            filters=filters[i], kernel_size=kernel_sizes[i],
            padding=padding, activation=activation,  kernel_initializer=kernel_initializer))(block)
            print(block.shape)
            blocks.append(block)

block = concatenate(blocks, axis=-1)
#for i in range(len(filters)):
#            block = Conv1D(
#            filters=filters[i], kernel_size=kernel_size[i],
#            padding=padding, activation=activation, strides = 1, kernel_initializer=kernel_initializer)(embed_char_out)
print(block.shape)
block = Dropout(dropout)(block)
block = TimeDistributed(MaxPooling1D(52))(block)
print("after max pooling", block.shape)

block = TimeDistributed(GlobalMaxPool1D())(block)
print("after global max pooling",block.shape)

char =  TimeDistributed(Flatten())(block)
print(char.shape)
char = Dropout(0.2)(char)
##########


(None, None, 52, 100)
(None, None, 52, 33)
(None, None, 52, 33)
(None, None, 52, 34)
(None, None, 52, 100)
after max pooling (None, None, 1, 100)
after global max pooling (None, None, 100)
(None, None, 100)


In [ ]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
pos_input= Input(shape=(None,), dtype='int32', name='pos_input')

In [ ]:
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
pos = Embedding(output_dim=POSEmbeddings.shape[1], input_dim=POSEmbeddings.shape[0], weights=[POSEmbeddings], trainable=False)(pos_input)


In [ ]:
#Bidirectional LSTM with Attention Layer Training
# Query-value attention of shap                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   e [batch_size, Tq, filters].
# query_value_attention_seq = tf.keras.layers.Attention()([query_seq_encoding, value_seq_encoding])
#import tensorflow.keras.backend as K
from tensorflow.keras.layers import Attention #Attention lauer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

#from crf import CRF
# using Attention layer from keras
#words_char_attention = Attention()([words,pos, char])
#words_char_attention = Attention()([words, char])
#print(words_char_attention.shape)
#words_char_attention = GlobalMaxPool1D()(words_char_attention_seq)

#print(words_char_attention.shape)

#attn_output= TimeDistributed(Attention()(output))#using Keras defined Attention layer
#print(output.shape)
'''
# attention layer using custom class
attn_output= attention(return_sequences=True)(output)
output = Dropout(0.5)(attn_output)
output=TimeDistributed(Flatten())(output)

print(output.shape)
'''



'\n# attention layer using custom class\nattn_output= attention(return_sequences=True)(output)\noutput = Dropout(0.5)(attn_output)\noutput=TimeDistributed(Flatten())(output)\n\nprint(output.shape)\n'

In [ ]:
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
class attention(Layer):

    def _init_(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(Attention,self)._init_()

    def build(self, input_shape):

        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        print(self.W.shape())
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")

        super(Attention,self).build(input_shape)

    def call(self, x):

        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a

        if self.return_sequences:
            return output

        return K.sum(output, axis=1)

In [ ]:
# concatenating the word and character embedding outputs
'''
#from keras_self_attention import SeqSelfAttention, SeqWeightedAttention

#output = concatenate([words,pos, char, words_char_attention, casing])
output = concatenate([words,pos, char, casing])  #without attention layer
print(output.shape)
output = Bidirectional(LSTM(100, return_sequences=True, dropout=0.2, recurrent_dropout=0.25))(output)

#attn_output = Attention()([words, char])
output = Attention(use_scale=True)([output,output]) #SELF-ATTENTION LAYER
#print(output.shape)
#output = Bidirectional(LSTM(100, return_sequences=True, dropout=0.2, recurrent_dropout=0.25))(output)
# fully connected DNN layer
output = TimeDistributed(Dense(150, activation = 'relu')) (output)

# output layer
#output = TimeDistributed(Dense(len(label2Idx), activation='softmax'))(output)
output = TimeDistributed(Dense(len(label2Idx)))(output)

#model = Model(inputs=[words_input, pos_input, casing_input,character_input], outputs=[output])
#model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer='nadam')
#model.summary()

#from crf import CRF
crf_ = CRF(len(label2Idx))
#crf_ = CRF(len(label2Idx), sparse_target=True)
output = crf_(output)
model = Model(inputs=[words_input, pos_input, casing_input,character_input], outputs=[output])

early_stopping=EarlyStopping(monitor="val_loss", patience = 3)
#reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=2, min_lr=0.0001)
#opt=tf.keras.optimizers.Nadam(learning_rate=0.00001)
#opt=tf.keras.optimizers.SGD(learning_rate=0.001)
#opt=tf.keras.optimizers.Adam(learning_rate=0.001)
opt=tf.keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer=opt, loss=crf_.loss, metrics=[crf_.accuracy])
#model.compile(optimizer=Adam(learning_rate=0.001), loss=crf_.loss, metrics=[crf_.accuracy])
#model.compile(optimizer='rmsprop', loss=crf_.loss, metrics=[crf_.accuracy])

#model = Model(inputs=[words_input, pos_input, casing_input,character_input], outputs=[output])
#model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(learning_rate=0.001))
#model.summary()

'''

'\n#from keras_self_attention import SeqSelfAttention, SeqWeightedAttention\n\n#output = concatenate([words,pos, char, words_char_attention, casing])\noutput = concatenate([words,pos, char, casing])  #without attention layer\nprint(output.shape)\noutput = Bidirectional(LSTM(100, return_sequences=True, dropout=0.2, recurrent_dropout=0.25))(output)\n\n#attn_output = Attention()([words, char])\noutput = Attention(use_scale=True)([output,output]) #SELF-ATTENTION LAYER\n#print(output.shape)\n#output = Bidirectional(LSTM(100, return_sequences=True, dropout=0.2, recurrent_dropout=0.25))(output)\n# fully connected DNN layer\noutput = TimeDistributed(Dense(150, activation = \'relu\')) (output)\n\n# output layer\n#output = TimeDistributed(Dense(len(label2Idx), activation=\'softmax\'))(output)\noutput = TimeDistributed(Dense(len(label2Idx)))(output)\n\n#model = Model(inputs=[words_input, pos_input, casing_input,character_input], outputs=[output])\n#model.compile(loss=\'sparse_categorical_crossent

In [ ]:
'''### new experiment...............................................................
# concatenating the word and character embedding outputs

#from keras_self_attention import SeqSelfAttention, SeqWeightedAttention

#output = concatenate([words,pos, char, words_char_attention, casing])
output = concatenate([words,pos, char, casing])  #without attention layer
print(output.shape)
output = Bidirectional(LSTM(100, return_sequences=True, dropout=0.2, recurrent_dropout=0.25))(output)



#attn_output = Attention()([words, char])
output = Attention(use_scale=True)([output,output]) #SELF-ATTENTION LAYER
#IDCNN
output = Conv1D(filters=256, kernel_size=2, activation='relu', padding='same',
                            dilation_rate=1, kernel_regularizer='l2')(output)
output = Conv1D(filters=256, kernel_size=3, activation='relu', padding='same',
                            dilation_rate=1, kernel_regularizer='l2')(output)
output= Conv1D(filters=512, kernel_size=4, activation='relu', padding='same',
                            dilation_rate=2, kernel_regularizer='l2')(output)

#print(output.shape)
#output = Bidirectional(LSTM(100, return_sequences=True, dropout=0.2, recurrent_dropout=0.25))(output)
# fully connected DNN layer
#output = TimeDistributed(Dense(150, activation = 'relu')) (output)

# output layer
#output = TimeDistributed(Dense(len(label2Idx), activation='softmax'))(output)
output = TimeDistributed(Dense(len(label2Idx)))(output)

#model = Model(inputs=[words_input, pos_input, casing_input,character_input], outputs=[output])
#model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer='nadam')
#model.summary()

#from crf import CRF
crf_ = CRF(len(label2Idx))
#crf_ = CRF(len(label2Idx), sparse_target=True)
output = crf_(output)
model = Model(inputs=[words_input, pos_input, casing_input,character_input], outputs=[output])

early_stopping=EarlyStopping(monitor="val_loss", patience = 3)
#reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=2, min_lr=0.0001)
#opt=tf.keras.optimizers.Nadam(learning_rate=0.00001)
#opt=tf.keras.optimizers.SGD(learning_rate=0.001)
#opt=tf.keras.optimizers.Adam(learning_rate=0.001)
opt=tf.keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer=opt, loss=crf_.loss, metrics=[crf_.accuracy])
#model.compile(optimizer=Adam(learning_rate=0.001), loss=crf_.loss, metrics=[crf_.accuracy])
#model.compile(optimizer='rmsprop', loss=crf_.loss, metrics=[crf_.accuracy])

#model = Model(inputs=[words_input, pos_input, casing_input,character_input], outputs=[output])
#model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(learning_rate=0.001))
#model.summary()
'''


'### new experiment...............................................................\n# concatenating the word and character embedding outputs\n\n#from keras_self_attention import SeqSelfAttention, SeqWeightedAttention\n\n#output = concatenate([words,pos, char, words_char_attention, casing])\noutput = concatenate([words,pos, char, casing])  #without attention layer\nprint(output.shape)\noutput = Bidirectional(LSTM(100, return_sequences=True, dropout=0.2, recurrent_dropout=0.25))(output)\n\n\n\n#attn_output = Attention()([words, char])\noutput = Attention(use_scale=True)([output,output]) #SELF-ATTENTION LAYER\n#IDCNN\noutput = Conv1D(filters=256, kernel_size=2, activation=\'relu\', padding=\'same\',\n                            dilation_rate=1, kernel_regularizer=\'l2\')(output)\noutput = Conv1D(filters=256, kernel_size=3, activation=\'relu\', padding=\'same\',\n                            dilation_rate=1, kernel_regularizer=\'l2\')(output)\noutput= Conv1D(filters=512, kernel_size=4, acti

In [ ]:
'''####global context
weight_init = tf.compat.v1.truncated_normal_initializer(mean=0.0, stddev=0.02)
weight_regularizer = tf.keras.regularizers.L2(0.0001)

def conv(x, channels, kernel=4, stride=2, pad=0, pad_type='zero', use_bias=True, sn=False, scope='conv_0'):
    with tf.compat.v1.variable_scope(scope):
        if pad > 0:
            h = x.get_shape().as_list()[1]
            if h % stride == 0:
                pad = pad * 2
            else:
                pad = max(kernel - (h % stride), 0)

            pad_top = pad // 2
            pad_bottom = pad - pad_top
            pad_left = pad // 2
            pad_right = pad - pad_left

            if pad_type == 'zero':
                x = tf.pad(x, [[0, 0], [pad_top, pad_bottom], [pad_left, pad_right], [0, 0]])
            if pad_type == 'reflect':
                x = tf.pad(x, [[0, 0], [pad_top, pad_bottom], [pad_left, pad_right], [0, 0]], mode='REFLECT')

        if sn:
            w = tf.compat.v1.get_variable("kernel", shape=[kernel, kernel, x.get_shape()[-1], channels], initializer=weight_init,
                                regularizer=weight_regularizer)
            x = Conv1D( filters=32, stride=1, padding='VALID',kernel_size=kernel)(x)
            if use_bias:
                bias = tf.get_variable("bias", [channels], initializer=tf.constant_initializer(0.0))
                x = tf.nn.bias_add(x, bias)

        else:
            x = tf.compat.v1.layers.conv1d(inputs=x, filters=channels,
                                 kernel_size=kernel, kernel_initializer=weight_init,
                                 kernel_regularizer=weight_regularizer,
                                 stride=stride, use_bias=use_bias)

        return x

def global_context_block(x, channels, use_bias=True, sn=False, scope='gc_block'):
    with tf.compat.v1.variable_scope(scope):
        with tf.compat.v1.variable_scope('context_modeling'):
            bs, h, w = x.get_shape().as_list()
            #c=1
            #x = tf.expand_dims(x, axis=-1)
            print(x.shape)
            input_x = x
            #input_x = hw_flatten(input_x)  # [N, H*W, C]
            input_x = tf.transpose(input_x, perm=[0, 2, 1])
            input_x = tf.expand_dims(input_x, axis=1)

            context_mask = conv(x, channels=1, kernel=1, stride=1, use_bias=use_bias, sn=sn, scope='conv')
            context_mask = hw_flatten(context_mask)
            context_mask = tf.nn.softmax(context_mask, axis=1)  # [N, H*W, 1]
            context_mask = tf.transpose(context_mask, perm=[0, 2, 1])
            context_mask = tf.expand_dims(context_mask, axis=-1)

            context = tf.matmul(input_x, context_mask)
            context = tf.reshape(context, shape=[bs, 1, 1, c])

        with tf.variable_scope('transform_0'):
            context_transform = conv(context, channels, kernel=1, stride=1, use_bias=use_bias, sn=sn, scope='conv_0')
            context_transform = layer_norm(context_transform)
            context_transform = relu(context_transform)
            context_transform = conv(context_transform, channels=c, kernel=1, stride=1, use_bias=use_bias, sn=sn, scope='conv_1')
            context_transform = sigmoid(context_transform)

            x = x * context_transform

        with tf.variable_scope('transform_1'):
            context_transform = conv(context, channels, kernel=1, stride=1, use_bias=use_bias, sn=sn, scope='conv_0')
            context_transform = layer_norm(context_transform)
            context_transform = relu(context_transform)
            context_transform = conv(context_transform, channels=c, kernel=1, stride=1, use_bias=use_bias, sn=sn, scope='conv_1')

            x = x + context_transform

        return x

def layer_norm(x, scope='layer_norm'):
    return tf.contrib.layers.layer_norm(x,
                                        center=True, scale=True,
                                        scope=scope)
def relu(x):
    return tf.nn.relu(x)

def sigmoid(x):
    return tf.sigmoid(x)

def hw_flatten(x):
    return tf.reshape(x, shape=[x.shape[0], -1, x.shape[-1]])

def spectral_norm(w, iteration=1):
    w_shape = w.shape.as_list()
    w = tf.reshape(w, [-1, w_shape[-1]])

    u = tf.compat.v1.get_variable("u", [1, w_shape[-1]], initializer=tf.random_normal_initializer(), trainable=False)

    u_hat = u
    v_hat = None
    for i in range(iteration):
        """
        power iteration
        Usually iteration = 1 will be enough
        """
        v_ = tf.matmul(u_hat, tf.transpose(w))
        v_hat = tf.nn.l2_normalize(v_)

        u_ = tf.matmul(v_hat, w)
        u_hat = tf.nn.l2_normalize(u_)

    u_hat = tf.stop_gradient(u_hat)
    v_hat = tf.stop_gradient(v_hat)

    sigma = tf.matmul(tf.matmul(v_hat, w), tf.transpose(u_hat))

    with tf.control_dependencies([u.assign(u_hat)]):
        w_norm = w / sigma
        w_norm = tf.reshape(w_norm, w_shape)

    return w_norm'''

'####global context\nweight_init = tf.compat.v1.truncated_normal_initializer(mean=0.0, stddev=0.02)\nweight_regularizer = tf.keras.regularizers.L2(0.0001)\n\ndef conv(x, channels, kernel=4, stride=2, pad=0, pad_type=\'zero\', use_bias=True, sn=False, scope=\'conv_0\'):\n    with tf.compat.v1.variable_scope(scope):\n        if pad > 0:\n            h = x.get_shape().as_list()[1]\n            if h % stride == 0:\n                pad = pad * 2\n            else:\n                pad = max(kernel - (h % stride), 0)\n\n            pad_top = pad // 2\n            pad_bottom = pad - pad_top\n            pad_left = pad // 2\n            pad_right = pad - pad_left\n\n            if pad_type == \'zero\':\n                x = tf.pad(x, [[0, 0], [pad_top, pad_bottom], [pad_left, pad_right], [0, 0]])\n            if pad_type == \'reflect\':\n                x = tf.pad(x, [[0, 0], [pad_top, pad_bottom], [pad_left, pad_right], [0, 0]], mode=\'REFLECT\')\n\n        if sn:\n            w = tf.compat.v1

In [ ]:
### new experiment  GCN...............................................................
# concatenating the word and character embedding outputs

output = concatenate([words,pos, char, casing])  #without attention layer
output = Bidirectional(LSTM(100, return_sequences=True, dropout=0.2, recurrent_dropout=0.25))(output)

filter_nums = 128
output = tf.keras.Sequential([
                    tf.keras.layers.Conv1D(filters=filter_nums, kernel_size=2, activation='relu',
                                           padding='same', dilation_rate=1,kernel_regularizer='l2'),
                    tf.keras.layers.Conv1D(filters=filter_nums, kernel_size=3, activation='relu',
                                           padding='same', dilation_rate=1,kernel_regularizer='l2'),
                    tf.keras.layers.Conv1D(filters=filter_nums, kernel_size=4, activation='relu',
                                           padding='same', dilation_rate=2,kernel_regularizer='l2')])(output)


output = Attention(use_scale=True)([output,output]) #SELF-ATTENTION LAYER
output=Dropout(0.2)(output)

output = Bidirectional(LSTM(100, return_sequences=True, dropout=0.2, recurrent_dropout=0.25))(output)

output = Attention(use_scale=True)([output,output]) #SELF-ATTENTION LAYER
output=Dropout(0.2)(output)

# fully connected DNN layer
output = TimeDistributed(Dense(100, activation = 'relu')) (output)
output=Dropout(0.2)(output)
# output layer
#output = TimeDistributed(Dense(len(label2Idx), activation='softmax'))(output)
output = TimeDistributed(Dense(len(label2Idx)))(output)


#from crf import CRF
crf_ = CRF(len(label2Idx))
output = crf_(output)

model = Model(inputs=[words_input, pos_input, casing_input,character_input], outputs=[output])

early_stopping=EarlyStopping(monitor="val_loss", patience = 3)
#reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=2, min_lr=0.0001)
#opt=tf.keras.optimizers.Nadam(learning_rate=0.00001)
#opt=tf.keras.optimizers.SGD(learning_rate=0.001)
#opt=tf.keras.optimizers.Adam(learning_rate=0.001)

opt=tf.keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer=opt, loss=crf_.loss, metrics=[crf_.accuracy])
#model.compile(optimizer=Adam(learning_rate=0.001), loss=crf_.loss, metrics=[crf_.accuracy])
#model.compile(optimizer='rmsprop', loss=crf_.loss, metrics=[crf_.accuracy])

#model = Model(inputs=[words_input, pos_input, casing_input,character_input], outputs=[output])
#model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(learning_rate=0.001))
#model.summary()



11 (None, None, 128)
22 (None, None, 200)


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 char_input (InputLayer)     [(None, None, 52)]           0         []                            
                                                                                                  
 char_embedding (TimeDistri  (None, None, 52, 100)        11100     ['char_input[0][0]']          
 buted)                                                                                           
                                                                                                  
 time_distributed (TimeDist  (None, None, 52, 33)         13233     ['char_embedding[0][0]']      
 ributed)                                                                                         
                                                                                              

In [ ]:
#tf.config.run_functions_eagerly(True)

In [ ]:
#from keras.callbacks import CSVLogger

#csv_logger = CSVLogger("model_history_log.csv", append=True)

In [ ]:
#Train the model
epochs = 20

train_acc_average = []
train_loss_average = []

valid_acc_average = []
valid_loss_average = []

for epoch in range(epochs):
    print("Epoch %d/%d"%(epoch,epochs))
    a = Progbar(len(train_batch_len))
    #b = Progbar(len(valid_batch_len))
    train_acc = []
    train_loss = []
    valid_acc = []
    valid_loss = []
    for i,batch in enumerate(iterate_minibatches(train_batch,train_batch_len)):
        labels, tokens, pos, casing,char = batch
        if(len(labels)>1):
          history = model.fit([tokens,pos, casing,char], labels, validation_split=0.1, callbacks=[early_stopping],verbose=1,use_multiprocessing=True)
          #history = model.fit([tokens,char], labels, validation_split=0.1, callbacks=[early_stopping],verbose=1, use_multiprocessing=True)

        #model.train_on_batch([tokens,pos, casing,char], labels)
        #train_acc.append(acc)
        #train_loss.append(loss)
        a.update(i)

In [ ]:
import numpy as np


#Method to compute the accruarcy. Call predict_labels to get the labels for the dataset
def compute_f1(predictions, correct, idx2Label):
    label_pred = []
    for sentence in predictions:
        label_pred.append([idx2Label[element] for element in sentence])

    label_correct = []
    for sentence in correct:
        label_correct.append([idx2Label[element] for element in sentence])


    #print label_pred
    #print label_correct

    acc = sum(1 for (a, b) in zip(label_correct, label_pred) if a== b) /len(label_correct)
    prec = compute_precision(label_pred, label_correct)
    rec = compute_precision(label_correct, label_pred)

    f1 = 0
    if (rec+prec) > 0:
        f1 = 2.0 * prec * rec / (prec + rec);

    return prec, rec, acc, f1

def compute_precision(guessed_sentences, correct_sentences):
    assert(len(guessed_sentences) == len(correct_sentences))
    correctCount = 0
    count = 0


    for sentenceIdx in range(len(guessed_sentences)):
        guessed = guessed_sentences[sentenceIdx]
        correct = correct_sentences[sentenceIdx]
        assert(len(guessed) == len(correct))
        idx = 0
        while idx < len(guessed):
            if guessed[idx][0] == 'B': #A new chunk starts
                count += 1

                if guessed[idx] == correct[idx]:
                    idx += 1
                    correctlyFound = True

                    while idx < len(guessed) and guessed[idx][0] == 'I': #Scan until it no longer starts with I
                        if guessed[idx] != correct[idx]:
                            correctlyFound = False

                        idx += 1

                    if idx < len(guessed):
                        if correct[idx][0] == 'I': #The chunk in correct was longer
                            correctlyFound = False


                    if correctlyFound:
                        correctCount += 1
                else:
                    idx += 1
            else:
                idx += 1

    precision = 0
    if count > 0:
        precision = float(correctCount) / count

    return precision

In [ ]:
#tf.keras.backend.set_learning_phase(False)
def Return_list(pred,labels):
  label_=[]
  for sentence in labels:
   label_.append(sentence.flatten().argmax(axis=-1))

  #print(len(label_[0]))
  pred = pred.argmax(axis=-1)
  #l = labels.argmax(axis=1)

  return list(pred),label_

In [ ]:
# Use the following prediction function when the labels are using numbers
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):
        tokens, pos, casing,char, labels = data
        tokens = np.asarray([tokens])
        pos=np.asarray([pos])
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens,pos, casing,char], verbose=False)[0]
        pred = pred.argmax(axis=-1) #Predict the classes
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    b.update(i+1)
    return predLabels, correctLabels

In [ ]:
# Use the following prediction function when the labels are using the one hot encoding technique
def tag_dataset_(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,batch in enumerate(dataset):
        tokens, pos, casing,char, labels = batch

        tokens = np.asarray([tokens])
        pos=np.asarray([pos])
        casing = np.asarray([casing])
        char = np.asarray([char])

        pred = model.predict([tokens,pos, casing,char], verbose=False)[0]

        pred,label=Return_list(pred,labels)

        correctLabels.append(label)
        predLabels.append(pred)

        b.update(i)
    b.update(i+1)
    return predLabels, correctLabels

In [ ]:
#model.save("biLSTM_attention_crf_2.h5")

In [ ]:
#   Performance on test dataset
#model = load_model ("models/biLSTM_attention.h5")
#tf.keras.backend.set_learning_phase(False)
predLabels, correctLabels = tag_dataset_(test_batch)


In [ ]:
pre_test, rec_test, acc_test, f1_test= compute_f1(predLabels, correctLabels, idx2Label)
#print(pre_test, rec_test, f1_test)
print("Test-Data: Prec: %.3f, Rec: %.3f, Acc: %.3f, F1: %.3f" % (pre_test, rec_test, acc_test, f1_test))

# save correct labels & prediected labels
with open ('correct.txt', 'w') as p:
	p.write(str(correctLabels))

with open ('pred_biLSTM_attn.txt', 'w') as fp:
	fp.write(str(predLabels))

In [ ]:
## convert multidimentional arrays to one list for all words
#l = [arr.tolist() for arr in predLabels]
#c = [b.tolist() for b in correctLabels]
import itertools

p_l=list(itertools.chain.from_iterable(predLabels))
c_l=list(itertools.chain.from_iterable(correctLabels))

#from sklearn.metrics import multilabel_confusion_matrix
#multilabel_confusion_matrix(p_l, c_l)
from sklearn.metrics import classification_report
print(classification_report(c_l, p_l))

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
plt.figure(figsize=(30,20))
fx=sns.heatmap(confusion_matrix(c_l,p_l), annot=True, fmt=".2f",cmap="GnBu")
fx.set_title('Confusion Matrix \n');
fx.set_xlabel('\n Predicted Values\n')
fx.set_ylabel('Actual Values\n');

plt.show()

In [ ]:
'''
# confusion
def plot_confusion_matrix(predLabels, correctLabels, idx2Label):
  conf_mat = confusion_matrix(predLabels, correctLabels)
  plt.subplots(figsize=(10, 10))
  sns.heatmap(conf_mat,annot=True,xticklabels=idx2Label.Type.values,yticklabels=idx2Label.Type.values)
fig = plt.figure()
ax = fig.add_subplot(111)
res = ax.imshow(array(conf_mat), cmap=cm.jet, interpolation='nearest')
cb = fig.colorbar(res)
savefig("confmat.png", format="png")
'''
